In [1]:
from google.colab import drive
drive.mount('/content/drive')
training_file = '/content/drive/MyDrive/all/ML/Part4/kol.sher'
raw_text = open(training_file, 'r').read()
raw_text = raw_text.lower()


Mounted at /content/drive


In [2]:
print(raw_text[:200])

ز شعر قدر و بها یافتند اگر شعرا
منم که شعر ز من یافته است قدر و بها
به پیش نادان گر قدر من بود پنهان
به پیش دانا باشد مقام من پیدا
همی نشاید گفتن که تیره شد خورشید
اگر نیاید روشن به دیده اعمی
شگفت نیس


In [3]:
all_words = raw_text.split()
unique_words = list(set(all_words))
print(f'Number of unique words: {len(unique_words)}')


Number of unique words: 39065


In [4]:
n_chars = len(raw_text)
print(f'Total characters: {n_chars}')


Total characters: 2800509


In [5]:
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print(f'Total vocabulary (unique characters): {n_vocab}')
print(chars)


Total vocabulary (unique characters): 62
['\n', ' ', ',', '-', '=', '\xa0', 'ء', 'آ', 'أ', 'ؤ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'ي', 'ٖ', '١', 'ٰ', 'ٱ', 'ٻ', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', '۱', '۲', '۳', '\u200b', '\u200d', '\u200e', '–', '’', 'ﭘ', 'ﺠ', 'ﻨ', 'ﻩ', 'ﻪ']


In [6]:
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_index = dict((c, i) for i, c in enumerate(chars))
print(char_to_index)


{'\n': 0, ' ': 1, ',': 2, '-': 3, '=': 4, '\xa0': 5, 'ء': 6, 'آ': 7, 'أ': 8, 'ؤ': 9, 'ا': 10, 'ب': 11, 'ة': 12, 'ت': 13, 'ث': 14, 'ج': 15, 'ح': 16, 'خ': 17, 'د': 18, 'ذ': 19, 'ر': 20, 'ز': 21, 'س': 22, 'ش': 23, 'ص': 24, 'ض': 25, 'ط': 26, 'ظ': 27, 'ع': 28, 'غ': 29, 'ف': 30, 'ق': 31, 'ل': 32, 'م': 33, 'ن': 34, 'ه': 35, 'و': 36, 'ي': 37, 'ٖ': 38, '١': 39, 'ٰ': 40, 'ٱ': 41, 'ٻ': 42, 'پ': 43, 'چ': 44, 'ژ': 45, 'ک': 46, 'گ': 47, 'ی': 48, '۱': 49, '۲': 50, '۳': 51, '\u200b': 52, '\u200d': 53, '\u200e': 54, '–': 55, '’': 56, 'ﭘ': 57, 'ﺠ': 58, 'ﻨ': 59, 'ﻩ': 60, 'ﻪ': 61}


In [7]:
import numpy as np
seq_length = 160
n_seq = int(n_chars / seq_length)


In [8]:
X = np.zeros((n_seq, seq_length, n_vocab))
Y = np.zeros((n_seq, seq_length, n_vocab))


In [9]:
for i in range(n_seq):
  x_sequence = raw_text[i * seq_length : (i + 1) * seq_length]
  x_sequence_ohe = np.zeros((seq_length, n_vocab))
  for j in range(seq_length):
    char = x_sequence[j]
    index = char_to_index[char]
    x_sequence_ohe[j][index] = 1.
  X[i] = x_sequence_ohe
  y_sequence = raw_text[i * seq_length + 1 : (i + 1) * seq_length + 1] 
  y_sequence_ohe = np.zeros((seq_length, n_vocab))
  for j in range(seq_length):
    try:
      char = y_sequence[j]
    except:
      # just for ignoring out of range error
      print('final sentence')
    index = char_to_index[char]
    y_sequence_ohe[j][index] = 1.
  Y[i] = y_sequence_ohe


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses,optimizers
tf.random.set_seed(42)


In [11]:
hidden_units = 700
dropout = 0.4


In [12]:
batch_size = 100
n_epoch= 300

In [13]:
model = models.Sequential()
model.add(layers.LSTM(hidden_units, input_shape=(None, n_vocab), return_sequences=True, dropout=dropout))
model.add(layers.LSTM(hidden_units, return_sequences=True, dropout=dropout))
model.add(layers.TimeDistributed(layers.Dense(n_vocab, activation='softmax')))


In [14]:
optimizer = optimizers.RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [15]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 700)         2136400   
                                                                 
 lstm_1 (LSTM)               (None, None, 700)         3922800   
                                                                 
 time_distributed (TimeDistr  (None, None, 62)         43462     
 ibuted)                                                         
                                                                 
Total params: 6,102,662
Trainable params: 6,102,662
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping


In [17]:
file_path = "/content/drive/MyDrive/all/ML/Part4/weights/weights_epoch_{epoch:03d}_loss_{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')


In [18]:
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=50, verbose=1, mode='min')


In [19]:
def generate_text(model, gen_length, n_vocab, index_to_char):
  index = np.random.randint(n_vocab)
  y_char = [index_to_char[index]]
  X = np.zeros((1, gen_length, n_vocab))
  for i in range(gen_length):
    X[0, i, index] = 1.
    indices = np.argmax(model.predict( X[:, max(0, i - seq_length -1):i + 1, :])[0], 1)
    index = indices[-1]
    y_char.append(index_to_char[index])
  return ''.join(y_char)



In [20]:
class ResultChecker(Callback):
  def __init__(self, model, N, gen_length):
    self.model = model
    self.N = N
    self.gen_length = gen_length
    
  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.N == 0:
      result = generate_text(self.model, self.gen_length, n_vocab, index_to_char)
      print('\nMy War and Peace:\n' + result)


In [21]:
result_checker = ResultChecker(model, 10, 500)

In [24]:
model.load_weights("/content/drive/MyDrive/all/ML/Part4/weights/weights_epoch_202_loss_2.0341.hdf5")
model.fit(X, Y, batch_size=batch_size, verbose=1, initial_epoch=201, epochs=n_epoch, callbacks=[result_checker, checkpoint, early_stop])


Epoch 202/300
176/176 [==============================] - ETA: 0s - loss: 2.0360
Epoch 202: loss improved from inf to 2.03600, saving model to /content/drive/MyDrive/all/ML/Part4/weights/weights_epoch_202_loss_2.0360.hdf5
176/176 [==============================] - 43s 242ms/step - loss: 2.0360
Epoch 203/300
176/176 [==============================] - ETA: 0s - loss: 2.0339
Epoch 203: loss improved from 2.03600 to 2.03389, saving model to /content/drive/MyDrive/all/ML/Part4/weights/weights_epoch_203_loss_2.0339.hdf5
176/176 [==============================] - 47s 265ms/step - loss: 2.0339
Epoch 204/300
176/176 [==============================] - ETA: 0s - loss: 2.0342
Epoch 204: loss did not improve from 2.03389
176/176 [==============================] - 45s 257ms/step - loss: 2.0342
Epoch 205/300
176/176 [==============================] - ETA: 0s - loss: 2.0338
Epoch 205: loss improved from 2.03389 to 2.03385, saving model to /content/drive/MyDrive/all/ML/Part4/weights/weights_epoch_205_lo